# Exploratory Data Analysis using Python - A Case Study

*Analyzing responses from the Stack Overflow Annual Developer Survey 2020*

![](https://i.imgur.com/qXhHKqv.png)



The following topics are covered in this tutorial:

- Selecting and downloading a dataset
- Data preparation and cleaning
- Exploratory analysis and visualization
- Asking and answering interesting questions
- Summarizing inferences and drawing conclusions

## Introduction

In this tutorial, we'll analyze the StackOverflow developer survey dataset. The dataset contains responses to an annual survey conducted by StackOverflow. You can find the raw data & official analysis here: https://insights.stackoverflow.com/survey.

There are several options for getting the dataset into Jupyter:

- Download the CSV manually and upload it via Jupyter's GUI
- Use the `urlretrieve` function from the `urllib.request` to download CSV files from a raw URL
- Use a helper library, e.g., [`opendatasets`](https://github.com/JovianML/opendatasets), which contains a collection of curated datasets and provides a helper function for direct download.

We'll use the `opendatasets` helper library to download the files.

In [ ]:
# !pip install opendatasets --quiet

In [ ]:
# import opendatasets as od
import pandas as pd

In [ ]:
od.download('stackoverflow-developer-survey-2020')

Let's verify that the dataset was downloaded into the directory `stackoverflow-developer-survey-2020` and retrieve the list of files in the dataset.

In [ ]:
import os

You can through the downloaded files using the "File" > "Open" menu option in Jupyter. It seems like the dataset contains three files:

- `README.txt` - Information about the dataset
- `survey_results_schema.csv` - The list of questions, and shortcodes for each question
- `survey_results_public.csv` - The full list of responses to the questions 

Let's load the CSV files using the Pandas library. We'll use the name `survey_raw_df` for the data frame to indicate this is unprocessed data that we might clean, filter, and modify to prepare a data frame ready for analysis.

In [ ]:
import pandas as pd

In [ ]:
survey_raw_df = pd.read_csv('survey_results_public.csv')

In [ ]:
survey_raw_df.head()

The dataset contains over 64,000 responses to 60 questions (although many questions are optional). The responses have been anonymized to remove personally identifiable information, and each respondent has been assigned a randomized respondent ID.

Let's view the list of columns in the data frame. 

In [ ]:
survey_raw_df.columns

In [ ]:
survey_raw_df.MainBranch.unique()

It appears that shortcodes for questions have been used as column names. 

We can refer to the schema file to see the full text of each question. The schema file contains only two columns: `Column` and `QuestionText`. We can load it as Pandas Series with `Column` as the index and the  `QuestionText` as the value.

In [ ]:
schema_fname = 'survey_results_schema.csv'
schema_raw = pd.read_csv(schema_fname, index_col='Column').QuestionText

In [ ]:
schema_raw

We can now use `schema_raw` to retrieve the full question text for any column in `survey_raw_df`.

In [ ]:
schema_raw['CompFreq']

We've now loaded the dataset. We're ready to move on to the next step of preprocessing & cleaning the data for our analysis.

## Data Preparation & Cleaning

While the survey responses contain a wealth of information, we'll limit our analysis to the following areas:

- Demographics of the survey respondents and the global programming community
- Distribution of programming skills, experience, and preferences
- Employment-related information, preferences, and opinions

Let's select a subset of columns with the relevant data for our analysis.

In [ ]:
selected_columns = [
    # Demographics
    'Country',
    'Age',
    'Gender',
    'EdLevel',
    'UndergradMajor',
    # Programming experience
    'Hobbyist',
    'Age1stCode',
    'YearsCode',
    'YearsCodePro',
    'LanguageWorkedWith',
    'LanguageDesireNextYear',
    'NEWLearn',
    'NEWStuck',
    # Employment
    'Employment',
    'DevType',
    'WorkWeekHrs',
    'JobSat',
    'JobFactors',
    'NEWOvertime',
    'NEWEdImpt'
]

In [ ]:
len(selected_columns)

Let's extract a copy of the data from these columns into a new data frame `survey_df`. We can continue to modify further without affecting the original data frame.

In [ ]:
survey_df = survey_raw_df[selected_columns].copy()

In [ ]:
schema = schema_raw[selected_columns]

In [ ]:
# with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):
#     print(schema)

Let's view some basic information about the data frame.

In [ ]:
survey_df.shape

In [ ]:
survey_df.info()

In [ ]:
schema

In [ ]:
schema['JobSat']

In [ ]:
~survey_df['JobSat'].isnull()

In [ ]:
# import numpy as np

In [ ]:
# np.nan

In [ ]:
survey_df[~(survey_df['JobSat'].isnull())]['JobSat']

Most columns have the data type `object`, either because they contain values of different types or contain empty values (`NaN`). It appears that every column contains some empty values since the Non-Null count for every column is lower than the total number of rows (64461). We'll need to deal with empty values and manually adjust the data type for each column on a case-by-case basis. 

Only two of the columns were detected as numeric columns (`Age` and `WorkWeekHrs`), even though a few other columns have mostly numeric values. To make our analysis easier, let's convert some other columns into numeric data types while ignoring any non-numeric value. The non-numeric are converted to `NaN`.

In [ ]:
survey_df.head()

In [ ]:
schema['NEWEdImpt']

In [ ]:
survey_df.NEWEdImpt.unique()

In [ ]:
survey_df.Age1stCode.unique()

In [ ]:
survey_df.YearsCode.unique()

In [ ]:
schema['YearsCode']

In [ ]:
schema['YearsCodePro']

In [ ]:
survey_df.YearsCodePro.unique()

In [ ]:
survey_df['Age1stCode'] = pd.to_numeric(survey_df.Age1stCode, errors='coerce')
survey_df['YearsCode'] = pd.to_numeric(survey_df.YearsCode, errors='coerce')
survey_df['YearsCodePro'] = pd.to_numeric(survey_df.YearsCodePro, errors='coerce')

Let's now view some basic statistics about numeric columns.

In [ ]:
survey_df.describe()

In [ ]:
survey_df.info()

There seems to be a problem with the age column, as the minimum value is 1 and the maximum is 279. This is a common issue with surveys: responses may contain invalid values due to accidental or intentional errors while responding. A simple fix would be to ignore the rows where the age is higher than 100 years or lower than 10 years as invalid survey responses. We can do this using the `.drop` method, [as explained here](https://www.geeksforgeeks.org/drop-rows-from-the-dataframe-based-on-certain-condition-applied-on-a-column/).

In [ ]:
survey_df[(survey_df['Age'] < 5) | (survey_df['Age'] > 100)].shape

In [ ]:
survey_df[survey_df.Age < 5].index

In [ ]:
survey_df.drop(survey_df[survey_df.Age < 5].index, inplace=True)
survey_df.drop(survey_df[survey_df.Age > 100].index, inplace=True)

The same holds for `WorkWeekHrs`. Let's ignore entries where the value for the column is higher than 140 hours. (~20 hours per day).

In [ ]:
survey_df.drop(survey_df[survey_df.WorkWeekHrs > 140].index, inplace=True)

The gender column also allows for picking multiple options. We'll remove values containing more than one option to simplify our analysis.

In [ ]:
survey_df['Gender'].value_counts()

In [ ]:
survey_df.Gender.str.contains(';', na=False).sum()

In [ ]:
import numpy as np

In [ ]:
survey_df.where(~(survey_df.Gender.str.contains(';', na=False)), np.nan, inplace=True)

We've now cleaned up and prepared the dataset for analysis. Let's take a look at a sample of rows from the data frame.

In [ ]:
survey_df['Gender'].value_counts()

In [ ]:
survey_df.describe()

In [ ]:
survey_df.sample(5)

In [ ]:
survey_df.Gender.unique()

## Exploratory Analysis and Visualization

Before we ask questions about the survey responses, it would help to understand the respondents' demographics, i.e., country, age, gender, education level, employment level, etc. It's essential to explore these variables to understand how representative the survey is of the worldwide programming community. A survey of this scale generally tends to have some [selection bias](https://en.wikipedia.org/wiki/Selection_bias).

Let's begin by importing `matplotlib.pyplot` and `seaborn`.

In [ ]:
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

sns.set_style('darkgrid')
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (9, 5)
matplotlib.rcParams['figure.facecolor'] = '#00000000'

### Country

Let's look at the number of countries from which there are responses in the survey and plot the ten countries with the highest number of responses.

In [ ]:
schema.Country

In [ ]:
survey_df.Country.nunique()

We can identify the countries with the highest number of respondents using the `value_counts` method.

In [ ]:
survey_df.Country.value_counts()

In [ ]:
top_countries = survey_df.Country.value_counts().head(15)
top_countries

We can visualize this information using a bar chart.

In [ ]:
plt.figure(figsize=(12,6))
plt.xticks(rotation=75)
plt.title(schema.Country)
sns.barplot(x=top_countries.index, y=top_countries);

It appears that a disproportionately high number of respondents are from the US and India, probably because the survey is in English, and these countries have the highest English-speaking populations. We can already see that the survey may not be representative of the global programming community - especially from non-English speaking countries. Programmers from non-English speaking countries are almost certainly underrepresented.

**Exercise**:
Try finding the percentage of responses from English-speaking vs. non-English speaking countries. You can use [this list of languages spoken in different countries](https://github.com/JovianML/opendatasets/blob/master/data/countries-languages-spoken/countries-languages.csv).

**Exercise**

In [ ]:
od.download('countries-languages.csv')

In [ ]:
import os

In [ ]:
os.listdir('countries-languages.csv')

In [ ]:
country_lang_df = pd.read_csv('countries-languages-spoken/countries-languages.csv')

In [ ]:
country_lang_df

In [ ]:
print(country_lang_df.Country.nunique())
print(survey_df.Country.nunique())

In [ ]:
survey_df.Country.unique()

In [ ]:
survey_df_Country_Langs = pd.DataFrame(survey_df.Country.dropna().unique(), columns=['Country'])

In [ ]:
survey_df_Country_Langs

In [ ]:
# Sort by Countries so as to do element-wise comparison with 'country_lang_df' dataframe
survey_df_Country_Langs.sort_values(by='Country',inplace=True)
survey_df_Country_Langs.reset_index(drop=True)

In [ ]:
# Sort by Countries so as to do element-wise comparison with 'survey_df_Country_Langs' dataframe
country_lang_df.sort_values(by='Country',inplace=True)
country_lang_df.reset_index(drop=True)

In [ ]:
# Cleanup and rename countries in 'survey_df_Country_Langs' according to 'country_lang_df'
survey_df_Country_Langs.loc[survey_df_Country_Langs.Country == 'Brunei Darussalam', 'Country'] = 'Brunei'
survey_df_Country_Langs.loc[survey_df_Country_Langs.Country == 'Congo, Republic of the...', 'Country'] = 'Congo, Republic of'
survey_df_Country_Langs.loc[survey_df_Country_Langs.Country == 'Democratic Republic of the Congo', 'Country'] = 'Congo, Democratic Republic of the'
survey_df_Country_Langs.loc[survey_df_Country_Langs.Country == 'Lao People\'s Democratic Republic', 'Country'] = 'Laos'
survey_df_Country_Langs.loc[survey_df_Country_Langs.Country == 'Libyan Arab Jamahiriya', 'Country'] = 'Libya'
survey_df_Country_Langs.loc[survey_df_Country_Langs.Country == 'Micronesia, Federated States of...', 'Country'] = 'Micronesia'
survey_df_Country_Langs.loc[survey_df_Country_Langs.Country == 'North Korea', 'Country'] = 'Korea, North'
survey_df_Country_Langs.loc[survey_df_Country_Langs.Country == 'South Korea', 'Country'] = 'Korea, South'
survey_df_Country_Langs.loc[survey_df_Country_Langs.Country == 'Republic of Korea', 'Country'] = 'Korea, South'
survey_df_Country_Langs.loc[survey_df_Country_Langs.Country == 'Republic of Moldova', 'Country'] = 'Moldova'
survey_df_Country_Langs.loc[survey_df_Country_Langs.Country == 'Russian Federation', 'Country'] = 'Russia'
survey_df_Country_Langs.loc[survey_df_Country_Langs.Country == 'Saint Lucia', 'Country'] = 'St. Lucia'
survey_df_Country_Langs.loc[survey_df_Country_Langs.Country == 'Saint Vincent and the Grenadines', 'Country'] = 'St. Vincent and the Grenadines'
survey_df_Country_Langs.loc[survey_df_Country_Langs.Country == 'Syrian Arab Republic', 'Country'] = 'Syria'
survey_df_Country_Langs.loc[survey_df_Country_Langs.Country == 'The former Yugoslav Republic of Macedonia', 'Country'] = 'Macedonia'
survey_df_Country_Langs.loc[survey_df_Country_Langs.Country == 'Timor-Leste', 'Country'] = 'East Timor'
survey_df_Country_Langs.loc[survey_df_Country_Langs.Country == 'United Republic of Tanzania', 'Country'] = 'Tanzania'
survey_df_Country_Langs.loc[survey_df_Country_Langs.Country == 'Venezuela, Bolivarian Republic of...', 'Country'] = 'Venezuela'
survey_df_Country_Langs.loc[survey_df_Country_Langs.Country == 'Viet Nam', 'Country'] = 'Vietnam'

In [ ]:
sum(~(survey_df_Country_Langs.Country.isin(country_lang_df.Country)))

In [ ]:
# A check to see if the countries are present in both 'survey_df_Country_Langs' and 'country_lang_df' dataframes
survey_df_Country_Langs['IsLanguageAvailable'] = survey_df_Country_Langs.Country.isin(country_lang_df.Country)
survey_df_Country_Langs.reset_index(drop=True)

In [ ]:
# These countries are present in 'survey_df_Country_Langs' but not in 'country_lang_df'
survey_df_Country_Langs[~survey_df_Country_Langs['IsLanguageAvailable']]

In [ ]:
# Merge 'survey_df_Country_Langs' and 'country_lang_df' dataframes on 'Country' column
survey_df_Country_Langs = pd.merge(survey_df_Country_Langs, country_lang_df, on='Country', how='left')

In [ ]:
survey_df_Country_Langs

In [ ]:
# Rename 'Languages Spoken' column to 'LanguagesSpoken'
survey_df_Country_Langs.rename({'Languages Spoken': 'LanguagesSpoken'}, axis=1, inplace=True)
survey_df_Country_Langs

In [ ]:
# 'IsEnglishSpeakingCountry' column to mention if the Country uses English as one of its spoken language
survey_df_Country_Langs['IsEnglishSpeakingCountry'] = survey_df_Country_Langs["LanguagesSpoken"].str.contains("English", na=False)
survey_df_Country_Langs.reset_index(drop=True)

In [ ]:
# Cleanup and rename countries in 'survey_df' according to 'survey_df_Country_Langs' or 'country_lang_df'
survey_df.loc[survey_df.Country == 'Brunei Darussalam', 'Country'] = 'Brunei'
survey_df.loc[survey_df.Country == 'Congo, Republic of the...', 'Country'] = 'Congo, Republic of'
survey_df.loc[survey_df.Country == 'Democratic Republic of the Congo', 'Country'] = 'Congo, Democratic Republic of the'
survey_df.loc[survey_df.Country == 'Lao People\'s Democratic Republic', 'Country'] = 'Laos'
survey_df.loc[survey_df.Country == 'Libyan Arab Jamahiriya', 'Country'] = 'Libya'
survey_df.loc[survey_df.Country == 'Micronesia, Federated States of...', 'Country'] = 'Micronesia'
survey_df.loc[survey_df.Country == 'North Korea', 'Country'] = 'Korea, North'
survey_df.loc[survey_df.Country == 'South Korea', 'Country'] = 'Korea, South'
survey_df.loc[survey_df.Country == 'Republic of Korea', 'Country'] = 'Korea, South'
survey_df.loc[survey_df.Country == 'Republic of Moldova', 'Country'] = 'Moldova'
survey_df.loc[survey_df.Country == 'Russian Federation', 'Country'] = 'Russia'
survey_df.loc[survey_df.Country == 'Saint Lucia', 'Country'] = 'St. Lucia'
survey_df.loc[survey_df.Country == 'Saint Vincent and the Grenadines', 'Country'] = 'St. Vincent and the Grenadines'
survey_df.loc[survey_df.Country == 'Syrian Arab Republic', 'Country'] = 'Syria'
survey_df.loc[survey_df.Country == 'The former Yugoslav Republic of Macedonia', 'Country'] = 'Macedonia'
survey_df.loc[survey_df.Country == 'Timor-Leste', 'Country'] = 'East Timor'
survey_df.loc[survey_df.Country == 'United Republic of Tanzania', 'Country'] = 'Tanzania'
survey_df.loc[survey_df.Country == 'Venezuela, Bolivarian Republic of...', 'Country'] = 'Venezuela'
survey_df.loc[survey_df.Country == 'Viet Nam', 'Country'] = 'Vietnam'

In [ ]:
# Merge 'survey_df' and 'survey_df_Country_Langs' on 'Country' column
survey_df = pd.merge(survey_df, survey_df_Country_Langs, on='Country', how='left')
survey_df

In [ ]:
# Drop unwanted columns 'IsLanguageAvailable', 'LanguagesSpoken' from 'survey_df' dataframe
survey_df.drop(columns=['IsLanguageAvailable', 'LanguagesSpoken'], inplace=True)
survey_df

In [ ]:
survey_df.IsEnglishSpeakingCountry.value_counts()

In [ ]:
survey_df.IsEnglishSpeakingCountry.value_counts() * 100 / survey_df.IsEnglishSpeakingCountry.count()

In [ ]:
IsEnglishSpeakingCountry_pct = survey_df.IsEnglishSpeakingCountry.value_counts() * 100 / survey_df.IsEnglishSpeakingCountry.count()

plt.figure(figsize=(14, 6))
plt.title('Responses for English-speaking vs. non-English speaking countries')

g = sns.barplot(y=IsEnglishSpeakingCountry_pct, x=IsEnglishSpeakingCountry_pct.index)
g.set_xticks(range(len(IsEnglishSpeakingCountry_pct))) # <--- set the ticks first
g.set_xticklabels(['Non-English speaking countries','English speaking countries']);

### Age

The distribution of respondents' age is another crucial factor to look at. We can use a histogram to visualize it. 

In [ ]:
plt.figure(figsize=(12, 6))
plt.title(schema.Age)
plt.xlabel('Age')
plt.ylabel('Number of respondents')

plt.hist(survey_df.Age, bins=np.arange(5,100,5), color='purple');

It appears that a large percentage of respondents are 20-45 years old. It's somewhat representative of the programming community in general. Many young people have taken up computer science as their field of study or profession in the last 20 years.

**Exercise**: You may want to filter out responses by age (or age group) if you'd like to analyze and compare the survey results for different age groups. Create a new column called AgeGroup containing values like `Less than 10 years`, `10-18 years`, `18-30 years`, `30-45 years`, `45-60 years` and `Older than 60 years`. Then, repeat the analysis in the rest of this notebook for each age group.


**Exercise**

In [ ]:
age_group_conditions = [survey_df.Age.le(10),
              survey_df.Age.gt(10) & survey_df.Age.le(18),
             survey_df.Age.gt(18) & survey_df.Age.le(30),
             survey_df.Age.gt(30) & survey_df.Age.le(45),
             survey_df.Age.gt(45) & survey_df.Age.le(60),
             survey_df.Age.gt(50)]

age_group_values = ['Less than 10 years', '10-18 years', '18-30 years', '30-45 years', '45-60 years','Older than 60 years']

In [ ]:
survey_df['AgeGroup'] = np.select(age_group_conditions, age_group_values)

In [ ]:
plt.figure(figsize=(14, 6))
plt.title('Number of respondents by Age Group')
plt.xticks(rotation=75)
plt.xlabel('Age Group')
plt.ylabel('Number of respondents by Age Group')

sns.histplot(survey_df.AgeGroup[survey_df.AgeGroup != '0'], color='purple');

In [ ]:
ax = sns.histplot(survey_df.AgeGroup[survey_df.AgeGroup != '0'], color='purple');
ax.bar_label(ax.containers[0]);

### Gender

Let's look at the distribution of responses for the Gender. It's a well-known fact that women and non-binary genders are underrepresented in the programming community, so we might expect to see a skewed distribution here.

In [ ]:
schema.Gender

In [ ]:
gender_counts = survey_df.Gender.value_counts()
gender_counts

A pie chart would be a great way to visualize the distribution.

In [ ]:
plt.figure(figsize=(12,6))
plt.title(schema.Gender)
plt.pie(gender_counts, labels=gender_counts.index, autopct='%1.1f%%', startangle=180);

Only about 8% of survey respondents who have answered the question identify as women or non-binary. This number is lower than the overall percentage of women & non-binary genders in the programming community - which is estimated to be around 12%. 

**Exercise**: It would be interesting to compare the survey responses & preferences across genders. Repeat this analysis with these breakdowns. How do the relative education levels differ across genders? You may find this analysis on the [Gender Divide in Data Science](https://medium.com/datadriveninvestor/exploratory-data-analysis-eda-understanding-the-gender-divide-in-data-science-roles-9faa5da44f5b) useful.

**Exercise**

In [ ]:
sns.countplot(y=survey_df.EdLevel, hue=survey_df.Gender, palette="bright");

### Education Level

Formal education in computer science is often considered an essential requirement for becoming a programmer. However, there are many free resources & tutorials available online to learn programming. Let's compare the education levels of respondents to gain some insight into this. We'll use a horizontal bar plot here.

In [ ]:
sns.countplot(y=survey_df.EdLevel)
plt.xticks(rotation=75)
plt.title(schema.EdLevel);

It appears that well over half of the respondents hold a bachelor's or master's degree, so most programmers seem to have some college education. However, it's not clear from this graph alone if they hold a degree in computer science.

**Exercises**: The graph currently shows the number of respondents for each option. Can you modify it to show the percentage instead?

**Exercise**

In [ ]:
# Education by percentage
EdLevel_pct = survey_df.EdLevel.value_counts() * 100 / survey_df.EdLevel.count()

sns.barplot(x=EdLevel_pct,y=EdLevel_pct.index)
plt.xticks(rotation=75)
plt.title(schema.EdLevel);

Let's also plot undergraduate majors, but this time we'll convert the numbers into percentages and sort the values to make it easier to visualize the order.

In [ ]:
schema.UndergradMajor

In [ ]:
undergrad_pct = survey_df.UndergradMajor.value_counts() * 100 / survey_df.UndergradMajor.count()

sns.barplot(x=undergrad_pct, y=undergrad_pct.index)
plt.title(schema.UndergradMajor)
plt.ylabel(None);
plt.xlabel('Percentage');

It turns out that 40% of programmers holding a college degree have a field of study other than computer science - which is very encouraging. It seems to suggest that while a college education is helpful in general, you do not need to pursue a major in computer science to become a successful programmer.

**Exercises**: Analyze the `NEWEdImpt` column for respondents who hold some college degree vs. those who don't. Do you notice any difference in opinion?


**Exercise**

In [ ]:
print(schema.NEWEdImpt, "\n", schema.EdLevel)

In [ ]:
survey_df.EdLevel.value_counts()

In [ ]:
holdcollegedegree = ['Bachelor’s degree (B.A., B.S., B.Eng., etc.)',
                                            'Master’s degree (M.A., M.S., M.Eng., MBA, etc.)',
                                            'Associate degree (A.A., A.S., etc.)',
                                            'Other doctoral degree (Ph.D., Ed.D., etc.)',
                                            'Professional degree (JD, MD, etc.)']
nocollegedegree = ['Some college/university study without earning a degree',
                                    'Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)',
                                    'Primary/elementary school',
                                    'I never completed any formal education']

edlevel_conditions = [survey_df.EdLevel.isin(holdcollegedegree),
              survey_df.EdLevel.isin(nocollegedegree)]

edlevel_values = [True,False]

In [ ]:
survey_df['HoldCollegeDegree'] = np.select(edlevel_conditions, edlevel_values)

In [ ]:
survey_df.NEWEdImpt.value_counts()

In [ ]:
plt.title("Analyze important is a formal education")
sns.countplot(y=survey_df.NEWEdImpt, hue=survey_df.HoldCollegeDegree, data=survey_df, palette="bright");

### Employment

Freelancing or contract work is a common choice among programmers, so it would be interesting to compare the breakdown between full-time, part-time, and freelance work. Let's visualize the data from the `Employment` column.

In [ ]:
survey_df.Employment.value_counts()

In [ ]:
(survey_df.Employment.value_counts(normalize=True, ascending=True)*100).plot(kind='barh', color='g')
plt.title(schema.Employment)
plt.xlabel('Percentage');

It appears that close to 10% of respondents are employed part time or as freelancers.

**Exercise**: Add a new column `EmploymentType` containing the values `Enthusiast` (student or not employed but looking for work), `Professional` (employed full-time, part-time or freelancing), and `Other` (not employed or retired). For each of the graphs that follow, show a comparison between `Enthusiast` and `Professional`.



**Exercise**

In [ ]:
emp_type_enthusiast = ['Student', 'Not employed, but looking for work']

emp_type_professional = ['Employed full-time', 'Employed part-time', 'Independent contractor, freelancer, or self-employed']

emp_type_other = ['Not employed, and not looking for work', 'Retired']

emp_type_conditions = [survey_df.Employment.isin(emp_type_enthusiast),
                        survey_df.Employment.isin(emp_type_professional),
                        survey_df.Employment.isin(emp_type_other)]

emp_type_values = ["Enthusiast", "Professional", "Other"]

In [ ]:
survey_df["EmploymentType"] = np.select(emp_type_conditions, emp_type_values)

In [ ]:
survey_df 

In [ ]:
survey_df["EmploymentType"].unique()

The `DevType` field contains information about the roles held by respondents. Since the question allows multiple answers, the column contains lists of values separated by a semi-colon `;`, making it a bit harder to analyze directly.

In [ ]:
schema.DevType

In [ ]:
survey_df.DevType.value_counts()

Let's define a helper function that turns a column containing lists of values (like `survey_df.DevType`) into a data frame with one column for each possible option.

In [ ]:
def split_multicolumn(col_series):
    result_df = col_series.to_frame()
    options = []
    # Iterate over the column
    for idx, value  in col_series[col_series.notnull()].iteritems():
        # Break each value into list of options
        for option in value.split(';'):
            # Add the option as a column to result
            if not option in result_df.columns:
                options.append(option)
                result_df[option] = False
            # Mark the value in the option column as True
            result_df.at[idx, option] = True
    return result_df[options]

In [ ]:
dev_type_df = split_multicolumn(survey_df.DevType)

In [ ]:
dev_type_df

The `dev_type_df` has one column for each option that can be selected as a response. If a respondent has chosen an option, the corresponding column's value is `True`. Otherwise, it is `False`.

We can now use the column-wise totals to identify the most common roles.


In [ ]:
dev_type_totals = dev_type_df.sum().sort_values(ascending=False)
dev_type_totals

In [ ]:
#  dev_type_df.count()

As one might expect, the most common roles include "Developer" in the name. 


**Exercise**: Can you figure out what percentage of respondents work in roles related to data science?

**Exercise**

In [ ]:
# Can you figure out what percentage of respondents work in roles related to data science?
data_science_pct = (dev_type_totals[["Data or business analyst","Data scientist or machine learning specialist","Engineer, data"]].values.sum() / dev_type_df.shape[0]) * 100
data_science_pct

18% of the respondents work in roles related to data science

In [ ]:
dev_type_totals[["Data or business analyst","Data scientist or machine learning specialist","Engineer, data"]].values.sum()

## Asking and Answering Questions

We've already gained several insights about the respondents and the programming community by exploring individual columns of the dataset. Let's ask some specific questions and try to answer them using data frame operations and visualizations.

#### Q: What are the most popular programming languages in 2020? 

To answer, this we can use the `LanguageWorkedWith` column. Similar to `DevType`, respondents were allowed to choose multiple options here.

In [ ]:
survey_df.LanguageWorkedWith

First, we'll split this column into a data frame containing a column of each language listed in the options.

In [ ]:
languages_worked_df = split_multicolumn(survey_df.LanguageWorkedWith)

In [ ]:
languages_worked_df

It appears that a total of 25 languages were included among the options. Let's aggregate these to identify the percentage of respondents who selected each language.


In [ ]:
languages_worked_percentages = languages_worked_df.mean().sort_values(ascending=False) * 100
languages_worked_percentages

We can plot this information using a horizontal bar chart.

In [ ]:
plt.figure(figsize=(12, 12))
sns.barplot(x=languages_worked_percentages, y=languages_worked_percentages.index)
plt.title("Languages used in the past year");
plt.xlabel('count');

Perhaps unsurprisingly, Javascript & HTML/CSS comes out at the top as web development is one of today's most sought skills. It also happens to be one of the easiest to get started. SQL is necessary for working with relational databases, so it's no surprise that most programmers work with SQL regularly. Python seems to be the popular choice for other forms of development, beating out Java, which was the industry standard for server & application development for over two decades.


#### Q: Which languages are the most people interested to learn over the next year?

For this, we can use the `LanguageDesireNextYear` column, with similar processing as the previous one.

In [ ]:
languages_interested_df = split_multicolumn(survey_df.LanguageDesireNextYear)
languages_interested_percentages = languages_interested_df.mean().sort_values(ascending=False) * 100
languages_interested_percentages

In [ ]:
plt.figure(figsize=(12, 12))
sns.barplot(x=languages_interested_percentages, y=languages_interested_percentages.index)
plt.title("Languages people are intersted in learning over the next year");
plt.xlabel('count');

Once again, it's not surprising that Python is the language most people are interested in learning - since it is an easy-to-learn general-purpose programming language well suited for a variety of domains: application development, numerical computing, data analysis, machine learning, big data, cloud automation, web scraping, scripting, etc. We're using Python for this very analysis, so we're in good company!


#### Q:  Which are the most loved languages, i.e., a high percentage of people who have used the language want to continue learning & using it over the next year?

While this question may seem tricky at first, it's straightforward to solve using Pandas array operations. Here's what we can do:

- Create a new data frame `languages_loved_df` that contains a `True` value for a language only if the corresponding values in `languages_worked_df` and `languages_interested_df` are both `True`
- Take the column-wise sum of `languages_loved_df` and divide it by the column-wise sum of `languages_worked_df` to get the percentage of respondents who "love" the language
- Sort the results in decreasing order and plot a horizontal bar graph

In [ ]:
languages_loved_df = languages_worked_df & languages_interested_df

In [ ]:
languages_loved_percentages = (languages_loved_df.sum() * 100/ languages_worked_df.sum()).sort_values(ascending=False)

In [ ]:
plt.figure(figsize=(12, 12))
sns.barplot(x=languages_loved_percentages, y=languages_loved_percentages.index)
plt.title("Most loved languages");
plt.xlabel('count');

[Rust](https://www.rust-lang.org) has been StackOverflow's most-loved language for [four years in a row](https://stackoverflow.blog/2020/01/20/what-is-rust-and-why-is-it-so-popular/). The second most-loved language is TypeScript, a popular alternative to JavaScript for web development.

Python features at number 3, despite already being one of the most widely-used languages in the world. Python has a solid foundation, is easy to learn & use, has a large ecosystem of domain-specific libraries, and a massive worldwide community.


**Exercise:** What are the most dreaded languages, i.e., languages which people have used in the past year but do not want to learn/use over the next year. Hint: `~languages_interested_df`.

**Exercise**

In [ ]:
languages_dreaded_df = languages_worked_df & ~languages_interested_df

In [ ]:
languages_dreaded_percentages = (languages_dreaded_df.sum() * 100/ languages_worked_df.sum()).sort_values(ascending=False)

In [ ]:
plt.figure(figsize=(12, 12))
sns.barplot(x=languages_dreaded_percentages, y=languages_dreaded_percentages.index)
plt.title("Most dreaded languages");
plt.xlabel('count');

#### Q: In which countries do developers work the highest number of hours per week? Consider countries with more than 250 responses only.

To answer this question, we'll need to use the `groupby` data frame method to aggregate the rows for each country. We'll also need to filter the results to only include the countries with more than 250 respondents.

In [ ]:
countries_df = survey_df.groupby('Country')[['WorkWeekHrs']].mean().sort_values('WorkWeekHrs', ascending=False)

In [ ]:
countries_df.head(20)

In [ ]:
high_response_countries_df = countries_df.loc[survey_df.Country.value_counts() > 250].head(15)

In [ ]:
high_response_countries_df

The Asian countries like Iran, China, and Israel have the highest working hours, followed by the United States. However, there isn't too much variation overall, and the average working hours seem to be around 40 hours per week.


#### Q: How important is it to start young to build a career in programming?

Let's create a scatter plot of `Age` vs. `YearsCodePro` (i.e., years of coding experience) to answer this question.

In [ ]:
schema.YearsCodePro

In [ ]:
sns.scatterplot(x='Age', y='YearsCodePro', hue='Hobbyist', data=survey_df)
plt.xlabel("Age")
plt.ylabel("Years of professional coding experience");

You can see points all over the graph, which indicates that you can **start programming professionally at any age**. Many people who have been coding for several decades professionally also seem to enjoy it as a hobby.

We can also view the distribution of the `Age1stCode` column to see when the respondents tried programming for the first time.

In [ ]:
plt.title(schema.Age1stCode)
sns.histplot(x=survey_df.Age1stCode, bins=30, kde=True);

As you might expect, most people seem to have had some exposure to programming before the age of 40. However, but there are people of all ages and walks of life learning to code.



## Inferences and Conclusions

We've drawn many inferences from the survey. Here's a summary of a few of them:

- Based on the survey respondents' demographics, we can infer that the survey is somewhat representative of the overall programming community. However, it has fewer responses from programmers in non-English-speaking countries and women & non-binary genders.

- The programming community is not as diverse as it can be. Although things are improving, we should make more efforts to support & encourage underrepresented communities, whether in terms of age, country, race, gender, or otherwise.


- Although most programmers hold a college degree, a reasonably large percentage did not have computer science as their college major. Hence, a computer science degree isn't compulsory for learning to code or building a career in programming.

- A significant percentage of programmers either work part-time or as freelancers, which can be a great way to break into the field, especially when you're just getting started.

- Javascript & HTML/CSS are the most used programming languages in 2020, closely followed by SQL & Python.

- Python is the language most people are interested in learning - since it is an easy-to-learn general-purpose programming language well suited for various domains.

- Rust and TypeScript are the most "loved" languages in 2020, both of which have small but fast-growing communities. Python is a close third, despite already being a widely used language.

- Programmers worldwide seem to be working for around 40 hours a week on average, with slight variations by country.

- You can learn and start programming professionally at any age. You're likely to have a long and fulfilling career if you also enjoy programming as a hobby.


## Exercises

There's a wealth of information to be discovered using the survey, and we've barely scratched the surface. Here are some ideas for further exploration:

- Repeat the analysis for different age groups & genders, and compare the results
- Pick a different set of columns (we chose 20 out of 65) to analyze other facets of the data
- Prepare an analysis focusing on diversity - and identify areas where underrepresented communities are at par with the majority (e.g., education) and where they aren't (e.g., salaries)
- Compare the results of this year's survey with the previous years and identify interesting trends


## References and Future Work

Check out the following resources to learn more about the dataset and tools used in this notebook:

- Stack Overflow Developer Survey: https://insights.stackoverflow.com/survey
- Pandas user guide: https://pandas.pydata.org/docs/user_guide/index.html
- Matplotlib user guide: https://matplotlib.org/3.3.1/users/index.html
- Seaborn user guide & tutorial: https://seaborn.pydata.org/tutorial.html
- `opendatasets` Python library: https://github.com/JovianML/opendatasets

